In [12]:
#Imports
import torch
import torchvision
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [8]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [9]:
# Create a bidirectional LSTM
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

In [13]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                          shuffle=True)

In [14]:
# Initialize network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# Train Network
from tqdm import tqdm
for epoch in range(num_epochs):
    for data, targets in tqdm(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

100%|██████████| 938/938 [00:10<00:00, 93.53it/s]


In [23]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training dataset")
    else:
        print("Checking accuracy on test dataset")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = round(float(num_correct)/float(num_samples)*100, 2)
        print(f'Got {num_correct} / {num_samples} with accuracy \
                {acc}')

In [24]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training dataset


100%|██████████| 938/938 [00:06<00:00, 139.22it/s]


Got 59069 / 60000 with accuracy                 98.45
Checking accuracy on test dataset


100%|██████████| 157/157 [00:01<00:00, 143.08it/s]

Got 9826 / 10000 with accuracy                 98.26
